# Import libraries 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
from geopy.distance import geodesic
import folium

from sklearn.neighbors import BallTree


# Helper Functions

In [2]:
#Map all tracked movement based on lat and long of GPS data  

def map_tracked_movement(df, zoom_start=12):
    # Create a map centered on the mean latitude and longitude
    map_center = [df['Lat'].mean(), df['Long'].mean()]
    movement_on_map = folium.Map(location=map_center, zoom_start=zoom_start)

    # Add CircleMarkers for each data point
    for index, row in df.iterrows():
        folium.CircleMarker(location=[row['Lat'], row['Long']], radius=2, color='blue', fill=True, fill_color='blue').add_to(movement_on_map)
    
    # Display the map
    return movement_on_map

In [3]:
def map_lifts(df, zoom_start=12):

    # Create a map centered on the mean latitude and longitude
    map_center = [df[['base_latitude', 'top_latitude']].mean().mean(), df[['base_longitude', 'top_longitude']].mean().mean()]
    movement_on_map = folium.Map(location=map_center, zoom_start=zoom_start)

    # Add CircleMarkers for each data point
    for _, row in df.iterrows():
        base_location = [row['base_latitude'], row['base_longitude']]
        top_location = [row['top_latitude'], row['top_longitude']]
        
        folium.CircleMarker(location=base_location, radius=5, color='blue', fill=True, fill_color='blue').add_to(movement_on_map)
        folium.CircleMarker(location=top_location, radius=5, color='green', fill=True, fill_color='green').add_to(movement_on_map)
    
    # Display the map
    return movement_on_map



In [4]:
def reduce_sampling_rate(df):
    df.set_index('Timestamp', inplace=True)
    # Set the 'Timestamp' column as the DataFrame's index
    df_resampled = df.resample('S').first()
    # Reset the index to create a standard numerical index
    df_resampled.reset_index(inplace=True)

    # option 
    # export to CSV
    # df_resampled.to_csv('df_206_reduced_sample_rate.csv', index=False)

    return df_resampled

In [5]:
def convert_datetime(df):

    # Convert 'Timestamp' column to datetime format
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ns')

    return df

# Load and Clean Data

In [6]:
# new data from gilberto
file_131='/Users/ze/Documents/Coding/Projects/2024/alturos/data/raw/v5_20240210_093434_131m.csv'
file_100='/Users/ze/Documents/Coding/Projects/2024/alturos/data/raw/v5_20240211_113607_100m.csv'
file_18='/Users/ze/Documents/Coding/Projects/2024/alturos/data/raw/Natschen-2024-02-04_12-04-18.csv'
file_46='/Users/ze/Documents/Coding/Projects/2024/alturos/data/raw/Natschen-2024-02-04_12-01-46.csv'
file_32='/Users/ze/Documents/Coding/Projects/2024/alturos/data/raw/Andermatt_Gondelbahn_Gutsch-2024-02-04_11-58-32.csv'
file_57='/Users/ze/Documents/Coding/Projects/2024/alturos/data/processed/df_57_labeled_on_lift.csv'
file_310='/Users/ze/Documents/Coding/Projects/2024/alturos/data/processed/df_310_labeled_on_lift_v4.csv'

In [7]:
# Load data 
df_131 = pd.read_csv(file_131)
df_100 = pd.read_csv(file_100)
df_18=pd.read_csv(file_18)
df_46=pd.read_csv(file_46)
df_32=pd.read_csv(file_32)
df_57=pd.read_csv(file_57)
df_310=pd.read_csv(file_310)

In [8]:
#convert_datetime
df_131=convert_datetime(df_131)
df_100=convert_datetime(df_100)
df_18=convert_datetime(df_18)
df_46=convert_datetime(df_46)
df_32=convert_datetime(df_32)
df_57=convert_datetime(df_57)
df_310=convert_datetime(df_310)

In [9]:
# # reduce sampling rate 
# df_131=reduce_sampling_rate(df_131)
# df_100=reduce_sampling_rate(df_100)
# # df_18=reduce_sampling_rate(df_18)
# # df_46=reduce_sampling_rate(df_46)
# # df_32=reduce_sampling_rate(df_32)
# df_57=reduce_sampling_rate(df_57)
# df_310=reduce_sampling_rate(df_310)

# Inspect location of data 

In [10]:
# map_tracked_movement(df_57)

In [11]:
# map_tracked_movement(df_131)

In [12]:
# map_tracked_movement(df_100)

In [13]:
# map_tracked_movement(df_18)

In [14]:
# map_tracked_movement(df_46)

In [15]:
# map_tracked_movement(df_32)

In [16]:
# map_tracked_movement(df_310)

# Lifts database 


Data from here [Skiresort Info](https://www.skiresort.info/ski-resort/gemsstock-andermatt/ski-lifts/l87981/)

In [17]:
# Load lift data from the database
lifts_db = pd.DataFrame({  
    'ski_resort': ['Gemsstock-Andermatt'],
    'lift_id': [1],  
    'lift_name': ['Gurschen-Bahn (Andermatt-Gurschenalp)'],
    'lift_type': ['Aerial tramway/Reversible ropeway'],
    'number_of_persons':[60],
    'base_station(m)': [1444],
    'top_station(m)': [2212],
    'length(m)': [1822],
    'base_latitude': [46.63242],
    'base_longitude': [8.59182],
    'top_latitude': [46.61844],
    'top_longitude': [8.59836],
    'transit_time': ['0:03:20'],
    'speed(m/s)': [10],
    'construction_date': [1991]
})

In [18]:
lifts_db.head()

,ski_resort,lift_id,lift_name,lift_type,number_of_persons,base_station(m),top_station(m),length(m),base_latitude,base_longitude,top_latitude,top_longitude,transit_time,speed(m/s),construction_date
0,Gemsstock-Andermatt,1,Gurschen-Bahn (Andermatt-Gurschenalp),Aerial tramway/Reversible ropeway,60,1444,2212,1822,46.63242,8.59182,46.61844,8.59836,0:03:20,10,1991


In [19]:
new_entries = [
    {
    'ski_resort': 'Gemsstock-Andermatt',
    'lift_id': 2, 
    'lift_name': 'Gemsstock-Bahn (Gurschenalp-Gemsstock)',
    'lift_type': 'Aerial tramway/Reversible ropeway',
    'number_of_persons': 80,
    'base_station(m)': 2212,  
    'top_station(m)': 2961,  
    'length(m)': 2159,
    'base_latitude': 46.6183,   
    'base_longitude': 8.59869,  
    'top_latitude': 46.6023, 
    'top_longitude': 8.61161, 
    'transit_time': '0:04:00', 
    'speed(m/s)': 10, 
    'construction_date': 1990  
    },
    {
    'ski_resort': 'Gemsstock-Andermatt',
    'lift_id': 3, 
    'lift_name': 'Gurschen-Flyer (Gurschengrat)',
    'lift_type': 'High speed chairlift (detachable) with bubble',
    'number_of_persons': 6,
    'base_station(m)': 2015,  
    'top_station(m)': 2281,  
    'length(m)': 901,
    'base_latitude': 46.62462,  
    'base_longitude': 8.59945,  
    'top_latitude': 46.61684,
    'top_longitude': 8.59712, 
    'transit_time': '0:03:20', 
    'speed(m/s)': 5,
    'construction_date': 2015  
    },
    {
    'ski_resort': 'Gemsstock-Andermatt',
    'lift_id': 4,  
    'lift_name': 'Lutersee-Lift',
    'lift_type': 'T-bar',
    'number_of_persons': np.nan,
    'base_station(m)': 2026,  
    'top_station(m)': 2398,  
    'length(m)': 1152,
    'base_latitude': 46.62291,   
    'base_longitude': 8.60324,  
    'top_latitude': 46.6166, 
    'top_longitude': 8.61499, 
    'transit_time': '0:08:30', 
    'speed(m/s)': 2.5, 
    'construction_date': 1975
    },
        {
    'ski_resort': 'Andermatt/​Oberalp/​Sedrun',
    'lift_id': 5,  
    'lift_name': 'Gütsch-Express 1 (Andermatt–Nätschen)',
    'lift_type': 'Gondola lift (monocable circulating ropeway)',
    'number_of_persons': 8,
    'base_station(m)': 1441, 
    'top_station(m)': 1832, 
    'length(m)': 1080,
    'base_latitude': 46.63781,  
    'base_longitude': 8.59542,  
    'top_latitude': 46.64268,
    'top_longitude': 8.60712, 
    'transit_time': '0:03:20', 
    'speed(m/s)': 6, 
    'construction_date': 2017
    },
        {
    'ski_resort': 'Andermatt/​Oberalp/​Sedrun',
    'lift_id': 6, 
    'lift_name': 'Gütsch-Express 2 (Nätschen-Gütsch)',
    'lift_type': 'Gondola lift (monocable circulating ropeway)',
    'number_of_persons': 8,
    'base_station(m)': 1832,  
    'top_station(m)': 2367,  
    'length(m)': 1902,
    'base_latitude': 46.64268,  
    'base_longitude': 8.60712,  
    'top_latitude': 46.65618,
    'top_longitude': 8.62127, 
    'transit_time': '0:05:50', 
    'speed(m/s)': 6, 
    'construction_date': 2017
    },
        {
    'ski_resort': 'Andermatt/​Oberalp/​Sedrun',
    'lift_id': 7, 
    'lift_name': 'Gütsch-Flyer',
    'lift_type': 'High speed chairlift (detachable) with bubble',
    'number_of_persons': 6,
    'base_station(m)': 2160,  
    'top_station(m)': 2344,
    'length(m)': 598,
    'base_latitude': 46.6528,  
    'base_longitude': 8.62766,  
    'top_latitude': 46.65639,
    'top_longitude': 8.622, 
    'transit_time': '0:02:10', 
    'speed(m/s)': 5, 
    'construction_date': 2016
    },
        {
    'ski_resort': 'Andermatt/​Oberalp/​Sedrun',
    'lift_id': 8, 
    'lift_name': 'Lutersee-Flyer',
    'lift_type': 'High speed chairlift (detachable) with bubble',
    'number_of_persons': 6,
    'base_station(m)': 2135,  
    'top_station(m)': 2400,
    'length(m)': 881,
    'base_latitude': 46.65602,  
    'base_longitude': 8.63412,  
    'top_latitude': 46.66339, 
    'top_longitude':8.63733, 
    'transit_time': '0:03:10', 
    'speed(m/s)': 5, 
    'construction_date': 2017
    },
        {
    'ski_resort': 'Andermatt/​Oberalp/​Sedrun',
    'lift_id': 9, 
    'lift_name': 'Schneehüenerstock-Flyer',
    'lift_type': 'High speed chairlift (detachable) with bubble',
    'number_of_persons': 6,
    'base_station(m)': 2165,  
    'top_station(m)': 2600,
    'length(m)': 1053,
    'base_latitude': 46.661,  
    'base_longitude': 8.65226,  
    'top_latitude': 46.66954,  
    'top_longitude': 8.65565, 
    'transit_time': '0:03:50', 
    'speed(m/s)': 5, 
    'construction_date': 2017
    }
    # df_57 will not be added to db 
]

In [20]:
# Convert the dictionary to DataFrame
new_entries_df = pd.DataFrame(new_entries)

# Append the new entries to the lifts_db DataFrame
lifts_db = pd.concat([lifts_db, new_entries_df], ignore_index=True)
lifts_db.head(15)

,ski_resort,lift_id,lift_name,lift_type,number_of_persons,base_station(m),top_station(m),length(m),base_latitude,base_longitude,top_latitude,top_longitude,transit_time,speed(m/s),construction_date
0,Gemsstock-Andermatt,1,Gurschen-Bahn (Andermatt-Gurschenalp),Aerial tramway/Reversible ropeway,60.0,1444,2212,1822,46.63242,8.59182,46.61844,8.59836,0:03:20,10.0,1991
1,Gemsstock-Andermatt,2,Gemsstock-Bahn (Gurschenalp-Gemsstock),Aerial tramway/Reversible ropeway,80.0,2212,2961,2159,46.61830,8.59869,46.60230,8.61161,0:04:00,10.0,1990
2,Gemsstock-Andermatt,3,Gurschen-Flyer (Gurschengrat),High speed chairlift (detachable) with bubble,6.0,2015,2281,901,46.62462,8.59945,46.61684,8.59712,0:03:20,5.0,2015
3,Gemsstock-Andermatt,4,Lutersee-Lift,T-bar,NaN,2026,2398,1152,46.62291,8.60324,46.61660,8.61499,0:08:30,2.5,1975
4,Andermatt/​Oberalp/​Sedrun,5,Gütsch-Express 1 (Andermatt–Nätschen),Gondola lift (monocable circulating ropeway),8.0,1441,1832,1080,46.63781,8.59542,46.64268,8.60712,0:03:20,6.0,2017
5,Andermatt/​Oberalp/​Sedrun,6,Gütsch-Express 2 (Nätschen-Gütsch),Gondola lift (monocable circulating ropeway),8.0,1832,2367,1902,46.64268,8.60712,46.65618,8.62127,0:05:50,6.0,2017
6,Andermatt/​Oberalp/​Sedrun,7,Gütsch-Flyer,High speed chairlift (detachable) with bubble,6.0,2160,2344,598,46.65280,8.62766,46.65639,8.62200,0:02:10,5.0,2016
7,Andermatt/​Oberalp/​Sedrun,8,Lutersee-Flyer,High speed chairlift (detachable) with bubble,6.0,2135,2400,881,46.65602,8.63412,46.66339,8.63733,0:03:10,5.0,2017
8,Andermatt/​Oberalp/​Sedrun,9,Schneehüenerstock-Flyer,High speed chairlift (detachable) with bubble,6.0,2165,2600,1053,46.66100,8.65226,46.66954,8.65565,0:03:50,5.0,2017


# Mapping lifts_db

In [21]:
map_lifts(lifts_db)

# Export 


In [23]:
# export to CSV
name_of_csv='lifts_db_v0.1.csv'
lifts_db.to_csv(name_of_csv, index=False)